In [18]:
import numpy as np
import pandas as pd
import numexpr

In [19]:
cols = 4
rows = int(5e7)
A = pd.DataFrame(np.random.rand(rows,cols))
A.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000000 entries, 0 to 49999999
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
dtypes: float64(4)
memory usage: 1.5 GB


## Assignment

### df[]

In [20]:
def fun1(M):
    M['new'] =1
    return(M)
%timeit fun1(A) 

470 ms ± 171 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### df.assign

In [21]:
def fun2(M):
    return(M.assign(new = 1))
%timeit fun2(A) #

1.65 s ± 78.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### df.eval

In [22]:
# assign
def fun3(M):
    return(M.eval('new =1'))
%timeit fun3(A) # 


2.02 s ± 37.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Filter

### query


In [23]:
A = pd.DataFrame(np.random.rand(rows,cols))
A['group'] = np.random.randint(0,4,rows)
A.head()

,0,1,2,3,group
0,0.790093,0.149202,0.652038,0.975080,1
1,0.108749,0.019993,0.202309,0.117972,1
2,0.285863,0.904456,0.029920,0.478838,0
3,0.972224,0.009880,0.503374,0.003457,1
4,0.344750,0.140094,0.155985,0.585301,3


In [24]:
%timeit A.query('group == 1')

1.02 s ± 7.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
A.query('group == 1')._is_view

False

In [26]:
%timeit A[A['group']==1]

579 ms ± 5.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
A[A['group']==1]._is_view

False

In [28]:
B = A.set_index('group')
%timeit B.loc[1]

680 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
B.loc[1] # verifying that identical to query

,0,1,2,3
group,,,,
1,0.790093,0.149202,0.652038,0.975080
1,0.108749,0.019993,0.202309,0.117972
1,0.972224,0.009880,0.503374,0.003457
1,0.339143,0.455876,0.427886,0.971892
1,0.163973,0.686110,0.813946,0.415434
...,...,...,...,...
1,0.387997,0.617476,0.578729,0.128652
1,0.345594,0.872919,0.032477,0.459502
1,0.390411,0.654556,0.897343,0.872757


In [30]:
A = A.astype({'group':"category"})


In [31]:
%timeit A.query('group == 1')

1.42 s ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
A = A.astype({'group':str})


In [33]:
%timeit A.query('group == "1"')

2.34 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit A[A['group']==1]

2.6 s ± 80.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Filter on multiple columns

In [35]:
A = pd.DataFrame(np.random.rand(rows,cols))
A['group1'] = np.random.randint(0,4,rows)
A['group2'] = np.random.randint(0,4,rows)
A['group3'] = np.random.randint(0,4,rows)
A['group4'] = np.random.randint(0,4,rows)
A.head()

,0,1,2,3,group1,group2,group3,group4
0,0.715060,0.079057,0.146865,0.483524,2,0,2,2
1,0.620963,0.331784,0.089985,0.419977,3,2,3,0
2,0.174324,0.883727,0.913365,0.747337,1,0,0,0
3,0.155817,0.002392,0.683785,0.636138,3,2,0,2
4,0.523497,0.537296,0.732802,0.295025,2,0,3,1


In [36]:
%timeit A.query('(group1 == 0) and (group2 == 0) and (group3 == 0) and (group4 == 0)')

570 ms ± 4.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%timeit A[(A.group1==0) & (A.group2==0) & (A.group3==0) & (A.group4==0)]

224 ms ± 29.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Filtering on object dtypes

In [38]:
cols = 4
rows = int(5e6)
A = pd.DataFrame(np.random.rand(rows,cols))
A['object1'] = pd.Series(['a','b','c','d']).sample(rows, replace=True).values
A['object2'] = pd.Series(['a','b','c','d']).sample(rows, replace=True).values
A['object3'] = pd.Series(['a','b','c','d']).sample(rows, replace=True).values
A['object4'] = pd.Series(['a','b','c','d']).sample(rows, replace=True).values


In [39]:
%timeit A[(A.object1=='a') & (A.object2=='a') & (A.object3=='a') & (A.object4=='a')]

1.01 s ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%timeit A.query('(object1==\'a\') and (object2==\'a\') and (object3==\'a\') and (object4==\'a\')')

602 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Groupby

### groupby apply


In [41]:
cols = 1
A = pd.DataFrame(np.random.randn(rows,cols)).rename(columns={0:'value'})
base = np.arange(0,4)
A['group'] = np.tile(base, rows//len(base))
A.head(10)

,value,group
0,-0.177644,0
1,-1.375356,1
2,-0.874615,2
3,0.019944,3
4,0.960310,0
5,-0.158121,1
6,-2.119782,2
7,-0.727461,3
8,-1.531364,0
9,1.223796,1


In [42]:
%timeit  A.groupby('group').apply(lambda x:x['value'].max()-x['value'].min())

229 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%timeit  A.groupby('group')['value'].apply(lambda x:x.max()-x.min())

188 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%timeit  A.groupby('group')['value'].apply(np.ptp)

172 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
%timeit A.groupby('group').agg(np.ptp)

191 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
%timeit A.groupby('group').agg({'value':np.ptp})

177 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### agg without np.ptp

In [47]:
%timeit  A.groupby('group')['value'].agg(lambda x:x.max()-x.min())

189 ms ± 927 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
def my_ptp(A):
    Ag = A.groupby('group')
    return(Ag.max()-Ag.min())
%timeit  my_ptp(A)

90.6 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [49]:
def ptp1(x):
    return(x.max()-x.min())

%timeit  A.groupby('group')['value'].agg(ptp1)

217 ms ± 27.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit  A.groupby('group')['value'].agg(ptp1, engine='cython')

196 ms ± 8.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
# needs debugging
# %timeit f = A.groupby('group')['value'].agg(ptp1, engine='numba')

### pivot

In [52]:
%timeit  pd.pivot_table(A, values='value', index='group', aggfunc=np.ptp)

195 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### stack with rowwise operation?

- The stacking may be expensive but is done once. 
- The data also needs to be structured enough. May be padded with NAs if required. 

In [53]:
B=A
B['id']= B.groupby('group').cumcount()
B_stacked = B.set_index(['group','id']).unstack()
B_stacked.head()

value                                                              \
id      0         1         2         3         4         5         6         
group                                                                         
0     -0.177644  0.960310 -1.531364  0.725363 -1.652571 -0.944556 -0.142448   
1     -1.375356 -0.158121  1.223796  0.280249 -0.999894 -0.608120  0.895123   
2     -0.874615 -2.119782  0.099949  0.613193 -0.538672  2.020434  0.466020   
3      0.019944 -0.727461  1.231646 -1.501745  1.800069 -0.713026  1.610591   

                                     ...                                \
id      7         8         9        ...   1249990   1249991   1249992   
group                                ...                                 
0     -1.818574 -0.473249  0.131492  ...  2.736597 -1.241736  0.591101   
1      0.080154  1.391215 -1.018439  ... -1.224880 -0.249292 -0.654399   
2     -0.432164  0.845843 -0.043639  ...  1.015951 -0.518561 -1.624884   
3      0.242695 -0.382019 -0.783716  ...  1.049805 -0.452261 -0.494208   

                                                                             
id      1249993   1249994   1249995   1249996   1249997   1249998   1249999  
group                                                                        
0     -0.249507 -0.763951 -0.336212  1.168941 -0.566667 -0.961223 -1.368270  
1      0.434519  0.291383  0.694229  2.794424 -0.878539  1.283029  0.348305  
2      0.092847 -0.693063 -0.469995  0.820167 -1.563106 -1.013027  0.218124  
3     -0.170028  1.828070 -1.594659 -0.857613  0.403470 -0.587384 -1.065238  

[4 rows x 1250000 columns]

In [54]:
# this is terrible!!!
%timeit B_stacked.max(axis=1)-B_stacked.min(axis=1)

4.13 s ± 174 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
%timeit B_stacked.apply(lambda x:x.max()-x.min(), axis=1)

21.4 ms ± 241 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
%timeit B_stacked.agg(np.ptp, axis=1)

105 ms ± 481 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%timeit B_stacked.agg(ptp1, axis=1)

121 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%timeit B_stacked.apply(ptp1, axis=1)

21.4 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
C = B_stacked.T
%timeit C.max(axis=0)-C.min(axis=0)

group
0    9.848997
1    9.846293
2    9.749873
3    9.644163
dtype: float64

In [60]:
%timeit C.agg(np.ptp, axis=0)

6.89 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
%timeit C.apply(np.ptp, axis=0)

6.85 ms ± 72.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [179]:
%timeit C.agg(ptp1, axis=0)

20.5 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
